In [43]:
import pandas as pd

# Đường dẫn đến tệp dữ liệu CSV
file_path = 'NBR_Pixel_Values_Export.csv'

# Đọc tệp CSV vào DataFrame
data = pd.read_csv(file_path)
data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45173 entries, 0 to 45172
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   system:index  45173 non-null  object 
 1   NBR           45173 non-null  float64
 2   year          45173 non-null  int64  
 3   .geo          45173 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.4+ MB


In [44]:
data[['year_number', 'pixel_id']] = data['system:index'].str.split('_', expand=True)
data['year_number'] = data['year'].astype(int)
data['pixel_id'] = data['pixel_id'].astype(int)

# Chuyển đổi DataFrame để mỗi dòng là một pixel và mỗi cột là NBR của từng năm
pivot_df = data.pivot(index='pixel_id', columns='year_number', values='NBR')

# Đổi tên các cột để phản ánh năm theo thứ tự (Year_1, Year_2, ..., Year_38)
pivot_df.columns = [f'Year_{col}' for col in pivot_df.columns]

# Hiển thị kết quả
pivot_df.reset_index(inplace=True)
pivot_df.head()
pivot_df.to_csv('pivot_data.csv', index=False)

In [45]:
import xarray as xr
data = pivot_df.dropna()
years = [int(col.split('_')[1]) for col in data.columns if col.startswith("Year_")]
pixel_ids = data['pixel_id']

# Chuyển đổi DataFrame sang xarray.Dataset
ds = xr.Dataset(
    {
        "NBR": (["pixel_id", "year"], data.drop(columns="pixel_id").values)
    },
    coords={
        "pixel_id": pixel_ids,
        "time": years
    }
)

# Hiển thị Dataset
print(ds)

<xarray.Dataset>
Dimensions:   (pixel_id: 1365, year: 33, time: 33)
Coordinates:
  * pixel_id  (pixel_id) int32 0 1 2 3 4 5 6 ... 1359 1360 1361 1362 1363 1364
  * time      (time) int32 1985 1986 1987 1988 1989 ... 2013 2014 2015 2016 2017
Dimensions without coordinates: year
Data variables:
    NBR       (pixel_id, year) float64 -843.8 -806.3 -818.1 ... -788.2 -807.5
